In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("data/stores_sales_simple/stores_sales_simple.csv")
df.head()

,unit_id,time,location,size,sales,ads
0,0,2020-01-01,LA,small,62319.650534,ad 2
1,0,2020-01-02,LA,small,61118.764040,ad 2
2,0,2020-01-03,LA,small,52576.959980,ad 2
3,0,2020-01-04,LA,small,34248.010877,ad 2
4,0,2020-01-05,LA,small,24711.595623,ad 2


In [18]:
df[["location", "size"]].drop_duplicates().values

array([['LA', 'small'],
       ['New York', 'large'],
       ['LA', 'large'],
       ['LA', 'medium'],
       ['Boston', 'medium'],
       ['New York', 'small'],
       ['Boston', 'large']], dtype=object)

In [37]:

unit_column="unit_id"
time_column="time"
actions=["ads"]
metrics=["sales"]
covariates=["location", "size"]



In [10]:
from algorithms.snn import SNN

In [71]:
assert len(metrics) ==  1, "method can only support single metric for now"

# get tensor dimensions
units = df[unit_column].unique()
N = len(units)
timesteps = df[time_column].unique()
T = len(timesteps)
list_of_actions = df[actions].drop_duplicates().agg('-'.join, axis=1).values
I = len(list_of_actions)
actions_dict = dict(zip(list_of_actions, np.arange(I)))
units_dict = dict(zip(np.arange(N), units))
time_dict = dict(zip(np.arange(T), timesteps))

# init tensors
tensor = np.full([N, T, I], np.nan)
# get tensor values
metric_matrix = df.pivot(index = unit_column, columns= time_column, values= metrics[0]).values
df["intervention_assignment"] = df[actions].agg('-'.join, axis=1).map(actions_dict).values
intervention_assignment_matrix = df.pivot(index = unit_column, columns= time_column, values= "intervention_assignment").values
for action_idx in range(I):
    tensor[intervention_assignment_matrix == action_idx, action_idx] = metric_matrix[intervention_assignment_matrix == action_idx]

# tensor to matrix




2

time,2020-01-01,2020-01-02,2020-01-03,2020-01-04,2020-01-05,2020-01-06,2020-01-07,2020-01-08,2020-01-09,2020-01-10,...,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19
unit_id,,,,,,,,,,,,,,,,,,,,,
0,62319.650534,NaN,NaN,34248.010877,NaN,NaN,40750.304085,NaN,NaN,63429.804115,...,NaN,NaN,54397.406855,NaN,NaN,25849.373014,NaN,NaN,59185.875432,NaN
1,NaN,56812.588240,NaN,NaN,26689.153876,NaN,NaN,53088.413817,NaN,NaN,...,38790.116325,NaN,NaN,58429.583737,NaN,NaN,39891.312304,NaN,NaN,46197.368834
2,NaN,NaN,48529.828147,NaN,NaN,32253.529718,NaN,NaN,57895.764584,NaN,...,NaN,43674.966133,NaN,NaN,35896.406942,NaN,NaN,39211.165120,NaN,NaN
3,47790.137724,NaN,NaN,49870.773977,NaN,NaN,50100.435296,NaN,NaN,50013.193403,...,NaN,NaN,34458.221255,NaN,NaN,34504.799511,NaN,NaN,38893.262907,NaN
4,NaN,63476.977007,NaN,NaN,23008.820325,NaN,NaN,58508.656454,NaN,NaN,...,49154.940299,NaN,NaN,58025.007910,NaN,NaN,43410.970693,NaN,NaN,53590.317913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,NaN,40390.108828,NaN,NaN,34412.944396,NaN,NaN,43933.382292,NaN,...,NaN,56819.635133,NaN,NaN,45761.763894,NaN,NaN,67045.047989,NaN,NaN
96,59180.741998,NaN,NaN,28501.177959,NaN,NaN,35273.845040,NaN,NaN,59312.947081,...,NaN,NaN,73306.489223,NaN,NaN,54311.100936,NaN,NaN,43045.914130,NaN
97,NaN,60938.909069,NaN,NaN,15239.704759,NaN,NaN,54470.068835,NaN,NaN,...,52033.550469,NaN,NaN,59559.225657,NaN,NaN,33219.942608,NaN,NaN,64010.259029


In [32]:
units

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])